# Basic tutorial: tabular data
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. run benchmarks
 5. visualise results interactively

In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import plotly.express as px
import torch
from torch import nn
import pandas as pd

from sklearn import datasets

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data import TorchDataLoader

from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [16]:
writer = SummaryWriter()

# Create your dataset

In [17]:
dl = TorchDataLoader(name="DoubleTori")
dl_tr, dl_ts = dl.build_dataloader(batch_size=1)

## Define and train your model

In [18]:
from gdeep.pipeline import Pipeline

model = nn.Sequential(nn.Flatten(), FFNet(0, arch=[3, 50, 3]))


In [19]:
from torch.optim import SGD

print(model)
loss_fn = nn.CrossEntropyLoss()

pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)

# train the model
pipe.train(SGD, 5, batch_size=1, lr=0.01)



Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): FFNet(
    (layer0): Linear(in_features=3, out_features=50, bias=True)
    (layer1): Linear(in_features=50, out_features=3, bias=True)
  )
)
TOTAL EPOCHS  5
Epoch 1
-------------------------------
Training loss: 0.559652  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 15.0%,                 Avg loss: 0.153419 

Epoch 2
-------------------------------
Training loss: 0.554609  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 12.5%,                 Avg loss: 0.163336 

Epoch 3
-------------------------------
Training loss: 0.560260  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 10.5%,                 Avg loss: 0.171289 

Epoch 4
-------------------------------
Training loss: 0.599338  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 11.0%,                 Avg loss: 0.174916 

Epoch 5
-------------------------------
Training loss: 0.559

# Simply use interpretability tools

In [20]:
from gdeep.analysis.interpretability import Interpreter

#inter = Interpreter(model, writer, method="LayerIntegratedGradients")

#inter.interpret_text("I am writing about money", 0, prec.vocabulary, prec.tokenizer, layer="embedding")

inter = Interpreter(model)
inter.interpret_tabular(next(iter(dl_tr))[0], next(iter(dl_tr))[1]);


d:\anaconda3\envs\giotto-deep2\lib\site-packages\captum\_utils\gradient.py:53: UserWarning:

Input Tensor 0 did not already require gradients, required_grads has been set automatically.

d:\anaconda3\envs\giotto-deep2\lib\site-packages\captum\attr\_core\deep_lift.py:322: UserWarning:

Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished



# Extract inner data from your models

In [21]:
from gdeep.models import ModelExtractor

me = ModelExtractor(model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k,item.shape)


1.layer0.weight torch.Size([50, 3])
1.layer0.bias torch.Size([50])
1.layer1.weight torch.Size([3, 50])
1.layer1.bias torch.Size([3])


In [22]:
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=100)
    res.shape

Executing the decison boundary computations:


In [23]:
x = next(iter(dl_tr))[0]
list_activations = me.get_activations(x)
len(list_activations)


5

In [24]:
x, target = next(iter(dl_tr))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)

torch.Size([50, 3])
torch.Size([50])
torch.Size([3, 50])
torch.Size([3])


# Visualise activations and other topological aspects of your model

In [25]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
vs.plot_activations(x)
vs.plot_persistence_diagrams(x)


Sending the plots to tensorboard: 


In [26]:

vs.plot_decision_boundary()

Executing the decison boundary computations:


(tensor([[ 0.1244, -0.1223, -0.1673],
         [-0.3965, -0.0204,  0.1017],
         [-0.3789, -0.0673, -0.0546],
         ...,
         [ 0.5644, -0.0586,  0.2975],
         [ 0.8285, -0.6142, -0.1527],
         [-0.3372, -0.0134,  0.2418]]),
 None,
 None)

In [27]:
vs.betti_plot_layers((0, 1), x)

In [28]:
plt = vs.plot_interpreter_tabular(inter)